## Tutorial 28: Deep learning for images

The real benefit of neural networks are when they are applied to high-dimensional
tasks such as image classification. As I mentioned last time, deep neural networks
are able to perform automatic dimensionality reduction in the first sets of layers
and then prediction tasks in the upper layers, all at the same time.

For today, let's load in all of the libraries we will need for the tutorial.

In [ ]:
import wiki
import iplot
import wikitext

import numpy as np

import numpy as np
import matplotlib.pyplot as plt
import sklearn

from matplotlib import pyplot as plt

In [ ]:
assert wiki.__version__ >= 6
assert wikitext.__version__ >= 2
assert iplot.__version__ >= 3

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input, decode_predictions
from keras.models import Model
from keras.preprocessing import image

### VGG-19

Rather than building a model from scratch, which is hard and time consuming, let's
just grab an image processing model directly. The model here is called VGG19; it was
designed for a well-known image processing challenged known as ILSVRC. (Note: The
first time you run this code may take a few minutes because keras does not pre-install
the model itself).

In [ ]:
vgg19_full = VGG19(weights='imagenet')
vgg19_full.summary()

This is an impressively large neural network. It has over 143 million
parameters!

**What kind of input and output is expected in the model?** 

### Loading an image file

I've written a small function to load an image file from a URL. We need a bit
of boilerplate code because the `load_image` function will not load an image over
the internet directly (only from a local file). 

In [ ]:
def load_image(link, target_size=None):
    import requests
    import shutil
    import os
    
    _, ext = os.path.splitext(link)
    
    r = requests.get(link, stream=True)
    with open('temp.' + ext, 'wb') as f:
        r.raw.decode_content = True
        shutil.copyfileobj(r.raw, f)
        
    img = image.load_img('temp.' + ext, target_size=target_size)
    return image.img_to_array(img)

Let's test out the function by grabbing and image of a dog. 

In [ ]:
img_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/Golden_retriever_eating_pigs_foot.jpg/170px-Golden_retriever_eating_pigs_foot.jpg"
img = load_image(img_path, target_size=(224, 224))
img.shape

Notice that this is a three-dimensional array containing the red, green, and blue
pixel intensities. As requested, the image has been reformated to be 224-by-224 pixels
in size.

We can see the image in Python by calling the `imshow` function, but need to divide all
of the pixel intensities by 256.

In [ ]:
plt.imshow(img / 256)

We need to do some pre-processing to the image before sending it to keras:

In [ ]:
x = np.expand_dims(img, axis=0)
x = preprocess_input(x)
x.shape

Finally, let's run this image through the VGG19 model 

In [ ]:
probs = vgg19_full.predict(x)
probs[0].shape

To assist in using the VGG19 function, keras provides a helper function 
`decode_predictions` that converts the probabilities into the most likely
categories. How well does the function do here?

In [ ]:
decode_predictions(probs, top=15)

It correctly predicts that this is a golden retriever.

Please do not take for granted how amazing this result is... Something
like this would have been impossible to create less than a decade ago.

## Experimenting with VGG19 — Part I

For the rest of class, you are going to do some experimentation with the 
VGG19 model. To start, take a look at some of the categories available
for prediction in the challenge:

- http://image-net.org/challenges/LSVRC/2014/browse-synsets

You might also take a look at a summary paper about the collection:

- https://arxiv.org/pdf/1409.0575.pdf

Your task here is to select 12 different image categories, trying to pick
a range of object types, and seeing how well the model performs. Try to
use the paper (see figure 15) to pick at least one "difficult" category.

(Hint: I've copied the self-contained code for the Golden retriever code
below; copy this 12 times and just modify the url)

In [ ]:
img_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/Golden_retriever_eating_pigs_foot.jpg/170px-Golden_retriever_eating_pigs_foot.jpg"
img = load_image(img_path, target_size=(224, 224))
plt.imshow(img / 256)
x = np.expand_dims(img, axis=0)
x = preprocess_input(x)
decode_predictions(vgg19_full.predict(x), top=15)

## Experimenting with VGG19 — Part II

Now, the second experiment asks you to dust off your HTML parsing. Write
code that starts with a Wikipedia page and prints out the predicted categories
for each image (you can filter the images if you would like) on the page. 

Test it on something interesting, like the page about dogs. You can, but do
not need to, wrap it up as a function. 